<a href="https://www.kaggle.com/code/ridheshgoti/breast-cancer?scriptVersionId=116525979" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Importing the libraries

In [1]:
import sys
!{sys.executable} -m pip install dicomsdl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 3.2 MB/s eta 0:00:00


In [2]:
#!pip install /kaggle/input/rsnapacks/dicomsdl-0.109.1-cp36-cp36m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
#import pydicom as dicom
import dicomsdl as dicom
import tensorflow as tf
import tensorflow_io as tfio
from PIL import Image as im
import os
from tqdm import tqdm
import cv2
from PIL import Image
from joblib import Parallel, delayed
import gc
import random

# Reading the training dataset of breast cancer patients

In [4]:
datasetPath = '/kaggle/input/rsna-breast-cancer-detection/train.csv'
imgPath = '/kaggle/input/rsna-breast-cancer-detection/train_images/'

def crop(sideName, imgName):
    if sideName == 'L':
        colind=[]
        for r,row in enumerate(imgName):
            for c,col in enumerate(row):
                if col==0:
                    colind.append(c)
                    break
        crop_size = max(colind)
        imgName = imgName[0:512,0:crop_size]
        imgName = cv2.resize(imgName,(128,128))
        
    if sideName == 'R':
        colind=[]
        for r,row in enumerate(imgName):
            for c,col in enumerate(row):
                if col!=0:
                    colind.append(c)
                    break
        crop_size = min(colind)
        imgName = imgName[0:512,crop_size:512]
        imgName = cv2.resize(imgName,(128,128))
    
    return imgName    

    gc.collect()
    
def crop_reverse(sideName, imgName):
    if sideName == 'R':
        colind=[]
        for r,row in enumerate(imgName):
            for c,col in enumerate(row):
                if col==0:
                    colind.append(c)
                    break
        crop_size = max(colind)
        imgName = imgName[0:512,0:crop_size]
        imgName = cv2.resize(imgName,(128,128))
        
    if sideName == 'L':
        colind=[]
        for r,row in enumerate(imgName):
            for c,col in enumerate(row):
                if col!=0:
                    colind.append(c)
                    break
        crop_size = min(colind)
        imgName = imgName[0:512,crop_size:512]
        imgName = cv2.resize(imgName,(128,128))
    
    return imgName    

    gc.collect()
    

def img_process(i,filename,sides):
    
    #ds = dicom.dcmread(filename)
    ds = dicom.open(filename)
    ds = ds.pixelData()
    ds = cv2.normalize(ds, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    ds = cv2.resize(ds,(512,512))
    ds = np.where(ds >= 0.9999, 0,ds)
    
    try:
        ds = np.array(crop(sides[i], ds))   
    except:
        ds = np.array(crop_reverse(sides[i], ds))
    return ds 

    #train_data.loc[i,'img_data'] = [img_fin]
    #train_data.to_csv('/kaggle/working/training_img_data.csv') 
    
    gc.collect()

def dcmToPix(datasetPath, imgPath):
    dataset = pd.read_csv(datasetPath)
    dataset=dataset[:50]
    
    
    
    patient_ids = dataset['patient_id']
    image_ids = dataset['image_id']
    sides  = dataset['laterality']

    imgData = []
    for pi, ii, leng in zip(patient_ids, image_ids, range(len(patient_ids))):
        imgData.append(imgPath + str(pi) + '/' + str(ii) + '.dcm')

    dataset['img_data'] = " "
    
    result = Parallel(n_jobs=128)(\
    delayed(img_process)(i, fname, sides) for i, fname in zip(range(len(imgData)),tqdm(imgData))\
    )
    
    dataset['img_data'] = result
    dataset.to_pickle('img_data.pkl' )
    
    return result
    

In [5]:
imgData = dcmToPix(datasetPath, imgPath)

 98%|█████████▊| 49/50 [00:00<00:00, 1450.37it/s]


In [6]:
#with (open('/kaggle/working/img_data.pkl', "rb")) as openfile:
#     imgData = pickle.load(openfile)

In [7]:
datacancer = pd.read_csv(datasetPath)
imgDataFrame = {'cancer':datacancer['cancer'][:50], 'img_data':imgData}
imgData2 = pd.DataFrame(imgDataFrame)


In [8]:
imgData=imgData2

In [9]:
zero_array = np.zeros((128,128))
counter=0
for i,j in enumerate(tqdm(imgData['img_data'])):
    if np.array_equiv(j,zero_array) == True:
        counter+=1
        imgData = imgData.drop(i)
      
print(counter)
        

100%|██████████| 50/50 [00:00<00:00, 22557.30it/s]

0


In [10]:
imgdata_pos = imgData[imgData['cancer'] == 1]
imgdata_neg = imgData[imgData['cancer'] == 0]

imgdata_pos = imgdata_pos.sample(frac = 1)
imgdata_neg = imgdata_neg.sample(frac = 1)

imgdata_neg = imgdata_neg.sample(frac= 0.6)

frames = 30*[imgdata_pos]
frames.append(imgdata_neg)
imgdata_shuff = pd.concat(frames)
imgdata_shuff = imgdata_shuff.sample(frac=1)

In [11]:
def random_rotate(imgData):
    imgData = imgData.reshape(128,128)
    imgData = im.fromarray(imgData)
    imgData = np.asarray(imgData.rotate(random.randint(0, 360)))
    return imgData

In [12]:
imgDataList=[]
for j in tqdm(imgdata_shuff['img_data']):
    imgDataList.append(random_rotate(j))
imgdata_shuff['img_data'] = imgDataList

100%|██████████| 30/30 [00:00<00:00, 3263.71it/s]


In [13]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(imgdata_shuff, test_size=0.3, random_state=42, shuffle=True)


train_target = np.array(train['cancer'])
train_features=[]
for i in train['img_data']:
    train_features.append([i])
train_features=np.array(train_features)


from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
norm_features= []
for i in range(len(train_features)):
    norm_features.append(scaler.fit_transform(train_features[i][0]))
train_features=np.array(norm_features)

train_features = train_features.reshape(len(train_features),128,128,1)



In [14]:
from keras.models import Sequential
from keras.layers import LeakyReLU
from keras.layers import Dense, Conv2D, Flatten,MaxPooling2D, Dropout, GlobalMaxPooling2D
model = Sequential()
model.add(Conv2D(16, 5, activation = "relu", input_shape = (128,128,1)))
model.add(MaxPooling2D())
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.2))
model.add(Conv2D(32, 5, activation = "relu"))
model.add(MaxPooling2D())
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(64, 3, activation = "relu"))
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation = 'relu')) 
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(64, activation = 'relu')) 
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = tf.keras.optimizers.Adam(\
    learning_rate=0.00001), loss= 'binary_crossentropy', metrics=['accuracy'])



2023-01-16 17:47:19.420266: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [15]:
model.fit(train_features, train_target, validation_split=0.3,batch_size = 32,epochs=30)




2023-01-16 17:47:19.757864: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30
1/1 [==============================] - 2s 2s/step - loss: 0.5925 - accuracy: 1.0000 - val_loss: 0.6068 - val_accuracy: 1.0000
Epoch 2/30
1/1 [==============================] - 0s 136ms/step - loss: 0.5614 - accuracy: 1.0000 - val_loss: 0.5931 - val_accuracy: 1.0000
Epoch 3/30
1/1 [==============================] - 0s 139ms/step - loss: 0.5537 - accuracy: 0.9286 - val_loss: 0.5795 - val_accuracy: 1.0000
Epoch 4/30
1/1 [==============================] - 0s 153ms/step - loss: 0.5613 - accuracy: 0.9286 - val_loss: 0.5655 - val_accuracy: 1.0000
Epoch 5/30
1/1 [==============================] - 0s 176ms/step - loss: 0.5040 - accuracy: 0.9286 - val_loss: 0.5517 - val_accuracy: 1.0000
Epoch 6/30
1/1 [==============================] - 0s 135ms/step - loss: 0.4474 - accuracy: 1.0000 - val_loss: 0.5380 - val_accuracy: 1.0000
Epoch 7/30
1/1 [==============================] - 0s 219ms/step - loss: 0.4207 - accuracy: 1.0000 - val_loss: 0.5243 - val_accuracy: 1.0000
Epoch 8/30
1/1 [=======

In [16]:
test_features=[]
for i in test['img_data']:
    test_features.append([i])
test_features=np.array(test_features)
 


scaler = StandardScaler()
norm_features= []
for i in range(len(test_features)):
    norm_features.append(scaler.fit_transform(test_features[i][0]))
test_features=np.array(norm_features)


test_features = test_features.reshape(len(test_features),128,128,1)


test_target = np.array(test['cancer'])

In [17]:
pred = model.predict(test_features)
bin_pred = []
for i in pred:
    if i>=0.5:
        bin_pred.append(1)
    else:
        bin_pred.append(0)
bin_pred = np.array(bin_pred)


In [18]:
import sklearn
acc = sklearn.metrics.accuracy_score(test_target, bin_pred)
print(acc)

countzero=0
countone=0
countzerot=0
countonet=0
for i, j in zip(test_target, bin_pred):
    if i==0 and j==0:
        countzero+=1
    if i==1 and j==1:
        countone+=1
    if i==1:
        countonet+=1
    if i==0:
        countzerot+=1
#print(countzero/countzerot,countone/countonet)

1.0
